# Migration from Standard Logging

Step-by-step migration guide with before/after examples.

## Step 1: Before (Standard Logging)

In [ ]:
import logging
import sys

logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

# Your current code
def process_user_login(username, user_id, ip_address):
    logger.info(f"User {username} (ID: {user_id}) logged in from {ip_address}")

def handle_error(operation, error_msg, user_id=None):
    logger.error(f"Error in {operation} for user {user_id}: {error_msg}")

# Test current logging
process_user_login("alice", 123, "192.168.1.100")
handle_error("payment", "Card declined", 123)

**Problems:** Hard to search, no filtering, string parsing needed

## Step 2: Drop-in Replacement

In [ ]:
# Just change the import!
import logstructor

logger = logstructor.getLogger(__name__)

# Same functions work unchanged
process_user_login("alice", 123, "192.168.1.100")
handle_error("payment", "Card declined", 123)

print("✅ Existing code works unchanged!")

## Step 3: Add Structure

In [ ]:
# Enhanced with structured fields
def process_user_login_v2(username, user_id, ip_address):
    logger.info(
        "User logged in",
        user_id=user_id,
        username=username,
        ip_address=ip_address,
        event_type="login"
    )

def handle_error_v2(operation, error_msg, user_id=None, error_code=None):
    logger.error(
        "Operation failed",
        operation=operation,
        error_message=error_msg,
        user_id=user_id,
        error_code=error_code
    )

process_user_login_v2("alice", 123, "192.168.1.100")
handle_error_v2("payment", "Card declined", 123, "CC_DECLINED")

## Step 4: Add Context Management

In [ ]:
def simulate_web_request(request_id, user_id):
    # Set context once
    logstructor.bind_context(request_id=request_id, user_id=user_id)
    
    try:
        # All logs automatically include context
        json_logger.info("Request started")
        json_logger.info("Processing", step="validation")
        json_logger.info("Request completed", status_code=200)
    finally:
        logstructor.clear_context()

simulate_web_request("req-123", 456)

## Benefits After Migration

✅ **Searchable logs**: `user_id:123`  
✅ **Better debugging**: Context automatically included  
✅ **Business insights**: Query logs like a database  
✅ **Faster incident response**: Structured alerts  

**Query examples:**
```bash
# Find all actions by user 123
context.user_id:123

# Find all errors
level:ERROR

# Find payment errors
context.operation:"payment" AND level:ERROR
```